# all info exhibition


In [4]:
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')


threshold_value = 0.3 
out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta'

metrics = ['degree',
            'global_efficiency', 'clustering_coefficient',
             'transitivity']


for metric in metrics:
    subjects_values = []
    columns = []
    for subject in subjects:
        observations = subject.get_all_observation()
        values = []
        for obs in observations:
            value = obs.bold.get_global_metric(metric, mat_name='brant_post/4.0base360dsubnetwork_network.mat', #对应的NBS文件
                                               threshold_type='intensity',
                                               threshold_value=threshold_value)
            columns.append(obs.name)
            subjects_values.append(float(value))
            #print(obs.name)

    subjects_values = np.array(subjects_values)
    columns = np.array(columns)
    new_array = np.stack((subjects_values, columns), axis=-1)

    index = ['subject{}'.format(i//7) for i in range(7,70)]
    df = pd.DataFrame(data=new_array,
                        index=index,
                        columns=['Value', 'Observation'])
    df[['Value']] = df[['Value']].astype(float)

    # ax = sns.violinplot(x='Observation', y="Value", data=df)
    # ax = sns.lineplot(x='Observation', y="Value", data=df)
    ax = sns.lineplot(x='Observation', y="Value", data=df, err_style="bars") 
    
    ax.set_title('whole_network_{}'.format(metric))
    
    out_png_path = os.path.join(out_path, 'xxx_sub_{}.png'.format(metric)) 
    out_csv_path = os.path.join(out_path, 'xxx_sub_{}.csv'.format(metric)) 
    plt.savefig(out_png_path)
    plt.close()
    df.to_csv(out_csv_path)

# pair t test for graph-theory info

In [1]:
#网络指标
from statistical_analysis import ttest
from scipy.stats import ttest_rel
from PDdatasets.subject import load_subjects


subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')
threshold_type='intensity'
threshold_value = 0.3
mat_name = 'brant_post/4.0base360dsubnetwork_network.mat'

metrics = ['degree',
            'global_efficiency', 'clustering_coefficient',
             'transitivity']

obs_names = [('base', '001d'), ('base', '007d'), ('base', '030d'), ('base', '090d'), ('base', '360d')]

for obs_name1, obs_name2 in obs_names:
    for metric_name in metrics:
        v1 = ttest.load_subjects_global_metric(subjects, obs_name1, mat_name,
                                               metric_name, threshold_type, threshold_value)
        v2 = ttest.load_subjects_global_metric(subjects, obs_name2, mat_name,
                                               metric_name, threshold_type, threshold_value)
        t, p = ttest_rel(v1, v2)
        if p < 0.05:
            print('{}-{}:{}: t:{:.2f}, p:{:.2e}'.format(obs_name1, obs_name2, metric_name, t, p))

base-030d:neighbor_degree: t:2.98, p:1.76e-02
base-030d:betweenness_centrality: t:-2.98, p:1.77e-02
base-030d:degree: t:2.80, p:2.34e-02
base-030d:shortest_path_length: t:-2.44, p:4.07e-02
base-030d:global_efficiency: t:2.59, p:3.22e-02
base-030d:clustering_coefficient: t:3.23, p:1.21e-02
base-030d:local_efficiency: t:2.59, p:3.21e-02
base-030d:transitivity: t:2.96, p:1.81e-02


C:\Users\timkl\Anaconda3\envs\autumor\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\timkl\Anaconda3\envs\autumor\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\timkl\Anaconda3\envs\autumor\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\timkl\Anaconda3\envs\autumor\lib\site-packages\numpy\core\_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


base-360d:betweenness_centrality: t:-4.09, p:3.49e-03
base-360d:degree: t:4.08, p:3.51e-03
base-360d:global_efficiency: t:3.67, p:6.29e-03
base-360d:clustering_coefficient: t:5.57, p:5.29e-04
base-360d:local_efficiency: t:3.68, p:6.25e-03
base-360d:transitivity: t:5.42, p:6.33e-04


# correlation with clinical improvement

In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import ttest
from scipy.stats import ttest_rel
from statistical_analysis.ttest import load_clinical
from statistical_analysis import correlation
from PDutils import NBS_subnet_extract
import pandas as pd
import numpy as np
from statistical_analysis.ttest import load_subjects_global_metric

obs_name1 = 'base'
obs_name2 = '360d'
thres_type ='intensity'
intensity_thres = 0.3
out_dir = r'H:\006pd_DTI\temp1'

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

mat_name = 'brant_post/03basecombine_network.mat'


metrics = ['degree',
            'global_efficiency', 'clustering_coefficient',
             'transitivity']

c_names = ['handtremor','CRST_A','CRST_B', 'CRST_C', 'CRST_TOTAL' ]

for c_name in c_names:
    clinical_v1 = load_clinical(subjects, obs_name1, c_name)
    clinical_v2 = load_clinical(subjects, obs_name2, c_name)
    clinical_delta = clinical_v1 - clinical_v2

    for metric in metrics:
        metric_v1 = load_subjects_global_metric(subjects, obs_name1, mat_name, metric, thres_type, intensity_thres)
        metric_v2 = load_subjects_global_metric(subjects, obs_name2, mat_name, metric, thres_type, intensity_thres)
        metric_delta = metric_v1 - metric_v2

        r, p = correlation.correlation(clinical_delta, metric_delta)
        print('{}_{}: r:{:.4f} p:{:.4e}'.format(c_name, metric, r, p))
        if p < 0.05:
            correlation.correlation(clinical_delta, metric_delta, save=True, out_path=os.path.join(out_dir, '{}_{}'.format(metric, c_name)))
            #print(row_node, col_node, r, p)


In [ ]:
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd
from statistical_analysis import correlation

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')


threshold_value = 0.3 
out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta' 

#metrics = ['degree','global_efficiency', 'transitivity']

metrics = ['degree']

c_names = ['handtremor','CRSTA_total','CRST b_total','CRST C','CRST TOTAL']

for metric in metrics:
    for c_name in c_names:
        subjects_values = []
        clinical_values = []
        for subject in subjects:
            observations = subject.get_all_observation()
            for obs in observations:
                if obs.name != 'ncnc' and obs.name != '001d'and obs.name != '007d':
                    value = obs.bold.get_global_metric(metric, mat_name='brant_post/4.0base360dsubnetwork_network.mat', #对应的NBS文件
                                                    threshold_type='intensity', 
                                                    threshold_value=threshold_value)
                    subjects_values.append(float(value))
                    clinical_values.append(obs.args[c_name])

        r, p = correlation.correlation(subjects_values, clinical_values, show=False)

        _, ax = plt.subplots(figsize=(float(5), float(5)))
        ax = sns.regplot(x=subjects_values,y=clinical_values, robust=True,
                            ax=ax,  scatter=False)
        ax.set_title('r={:.2f}, p={:.2e}'.format(r, p), fontdict={'fontsize': 14})

        ax.set_ylim(bottom=0, top=80)
        ax.tick_params(axis='both', which='major', labelsize=14)
        plt.tight_layout()
        plt.show()

        print('{}-{}, r:{}, p:{}'.format(metric, c_name, r, p))


In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import ttest
from scipy.stats import ttest_rel
from statistical_analysis.ttest import load_clinical
from statistical_analysis import correlation, plsr
from PDutils import NBS_subnet_extract
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

from statistical_analysis.ttest import load_subjects_nodal_metric

obs_name1 = 'base'
obs_name2 = '030d'
intensity_thres = 0.3
mat_name = 'brant_post/03basecombine_network.mat'

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

"""
# Readin NBS defined subnetwork
NBS_path1 = r'H:\006pd_DTI\13_C_BOLD_NBS_NEW\60_SM_R_weight_it0.3___\ncbase____\weight_start_net.mat'
subnetwork1 = subnetwork_extract.readin_NBS_subnetwork(NBS_path1)
"""

# Readin txt defined subnetwork
subnetwork1 = np.loadtxt(r'H:\006pd_DTI\10_BN_nbs_bold\02_R_wi0.3\combine.txt', delimiter =',')
all_nodes, row_nodes, col_nodes = NBS_subnet_extract.extract_subnetwork_nodes(subnetwork1) # matlab中坐标是从1开始
all_nodes = all_nodes+1

metrics = ['degree',
            'global_efficiency', 'clustering_coefficient',
             'transitivity']

metric_v1, metric_v2 = load_subjects_nodal_metric(subjects, obs_name1, obs_name2, mat_name, 'degree', thres_type, intensity_thres)
ts, ps = ttest_rel(metric_v1, metric_v2, axis=1)

value_dict = dict(zip(all_nodes, ts))

resutls = plsr.plsr(value_dict, n_components=2,
                n_perm=1, n_boot=1,
                gene_path=r'H:\006pd_DTI\05_gene_analysis\01_AHBAexpression\expression.csv',
                out_csv_path=r'H:\006pd_DTI\temp1/plsr.csv',
                out_model_path=r'H:\006pd_DTI\temp1/plsr.pickle')
print(resutls.permres.pvals)


# nodal analysis

In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import correlation
import numpy as np
from statistical_analysis.ttest import load_subjects_nodal_metric
import os

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')
#out_dir = r"H:\006pd_DTI\14_nodal_analysis"

from statistical_analysis.ttest import load_clinical

obs_name1 = 'base'
obs_name2 = '360d'

mat_name = 'brant_post/wholenet_network.mat'
thres_type = 'intensity'
threshold_value = 0.3

c_names = ['handtremor','CRST_A','CRST_B', 'CRST_C', 'CRST_TOTAL' ]

metrics = ['degree','global_efficiency', 'clustering_coefficient']

for c_name in c_names:
    clinical_v1 = load_clinical(subjects, obs_name1, c_name)
    clinical_v2 = load_clinical(subjects, obs_name2, c_name)
    clinical_delta = clinical_v1 - clinical_v2

    for m_name in metrics:
        metric_v1, metric_v2 = load_subjects_nodal_metric(subjects, obs_name1, obs_name2, mat_name, m_name, thres_type, threshold_value)
        delta_metric = metric_v1 - metric_v2

        j = 1
        for v in delta_metric:
            r,p = correlation.correlation(clinical_delta, v)
            if p < 0.05:
                print('Clinical:{} obs:{}-{}, metric:{}, Node:{}, r:{:.2f}, p:{:.2e}'.format(c_name, obs_name1, obs_name2, m_name, j, r, p))
                #correlation.correlation(clinical_delta, v, save=True, out_path=os.path.join(out_dir, 'Clinical:{} obs:{}-{}, metric:{}, Node:{}, r:{:.2f}, p:{:.2e}'.format(c_name, obs_name1, obs_name2, m_name, j, r, p)))
            j += 1

